In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 

import os
from pathlib import Path

import plotly.offline as plty
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
from colorama import Fore, Back, Style, init

from tqdm import tqdm
tqdm.pandas()

In [2]:
!pip install -q pyspellchecker

In [3]:
PATH = Path('/kaggle/input/imdb-dataset-of-50k-movie-reviews/')

data = pd.read_csv(PATH / 'IMDB Dataset.csv')

In [4]:
data.info()

In [4]:
data.head(5)

### Mapping Sentiments to Numbers

In [5]:
data['sentiment'] = data['sentiment'].map({'positive' : 0, 'negative' : 1})

### Text Preprocessing

In [6]:
from spellchecker import SpellChecker
import spacy
import string, re

spell = SpellChecker()
PUNCT_TO_REMOVE = string.punctuation

def remove_punctuation(text):    
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))


def correct_spellings(text): 
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)


def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [7]:
def clean_text(text):
    
    # Lower Casing
    text = text.lower()
    
    # Remove url
    text = remove_urls(text) 
    
    # Remove html tags
    text = remove_html(text)
    
    # Removing @tags
    text = re.sub('@\w*','',text)
    
    # Removing Punctuations
    text = remove_punctuation(text)
    
    # Removing new lines
    text = re.sub('\\n',' ',text)
    
    # Correct spellings
    #text = correct_spellings(text)      
    
    return text

In [8]:
data["clean_text"] = data["review"].progress_apply(lambda text: clean_text(text))

In [50]:
data[['review','clean_text']].head(10)

In [44]:
string = ' '.join(data['clean_text'])

wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(string.lower())
fig = px.imshow(wordcloud)
fig.update_layout(title_text='Common words in comments')

### Sentiment Analysis

In [51]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def polarity(text):
    if type(text) == str:
        return SIA.polarity_scores(text)
    else:
        return 1000
    
SIA = SentimentIntensityAnalyzer()
data["polarity"] = data["clean_text"].progress_apply(polarity)

In [52]:
import plotly.graph_objects as go

neg_pol = [pols['neg'] for pols in data["polarity"] if type(pols) is dict]
neg_pol = list(filter((0.0).__ne__, neg_pol))

fig = go.Figure(go.Histogram(x=neg_pol, marker=dict(
            color='red')
    ))

fig.update_layout(xaxis_title="Negativity sentiment", title_text="Negativity sentiment", template="simple_white")
fig.show()

From the above plot, we can see that negative sentiment has a strong rightward (positive) skew, indicating that negativity is usually on the lower side. This suggests that most comments are not toxic or negative. In fact, the most common negativity value is around 0.14. Virtually no comments have a negativity greater than 0.8.

In [53]:
toxic = [x['neg'] for x in data.query("sentiment == 1")['polarity'] if type(x) == dict]
non_toxic = [x['neg'] for x in data.query("sentiment == 0")['polarity'] if type(x) == dict]

fig = ff.create_distplot(hist_data=[toxic, non_toxic],
                         group_labels=["Negative", "Positive"],
                         colors=["darkorange", "dodgerblue"], show_hist=False)

fig.update_layout(title_text="Negativity of Bad vs Good tweets", xaxis_title="Negativity", template="simple_white")
fig.show()

We can clearly see that Neg comments have a significantly greater negative sentiment than Postive comments (on average). The probability density of negativity peaks at around 0 for non-toxic comments, while the negativity for toxic comments at 0.12 . This suggests that a comment is very likely to be non-toxic if it has a negativity of 0.

In [54]:
pos_pol = [pols['pos'] for pols in data["polarity"] if type(pols) is dict]
pos_pol = list(filter((0.0).__ne__, pos_pol))

fig = go.Figure(go.Histogram(x=pos_pol, marker=dict(
            color='seagreen')
    ))

fig.update_layout(xaxis_title="Positivity sentiment", title_text="Positivity sentiment", template="simple_white")
fig.show()

Most of the texts are Neutral by Looking at the low postivity & negativity, Let's look at netral values to confirm this

In [55]:
neu_pol = [pols['neu'] for pols in data["polarity"] if type(pols) is dict]
neu_pol = list(filter((1.0).__ne__, neu_pol))

fig = go.Figure(go.Histogram(x=neu_pol, marker=dict(
            color='darkorange')
    ))

fig.update_layout(xaxis_title="Neutral sentiment", title_text="Neutral sentiment", template="simple_white")
fig.show()

From the above plot, we can see that the neutrality sentiment distribution has a strong leftward (negative) skew, which is in constrast to the negativity and positivity sentiment distributions. This indicates that the comments tend to be very neutral and unbiased in general. This also suggests that most comments are not highly opinionated and polarizing, meaning that most comments are non-toxic.

In [56]:
total_comments = data['sentiment'].count()
neg = data['sentiment'].value_counts().loc[1]
Sentiment = ['Negative','Positive']
count = [neg, total_comments-neg]

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "bar"}, {"type": "pie"}]])
fig.add_trace(go.Bar(x=Sentiment,y=count,text=count, marker_color=['#D9636B', '#64D9D1']),
             row=1, col=1)
fig.add_trace(go.Pie(labels=Sentiment, values=count, domain=dict(x=[0.5, 1.0]), marker_colors=['#D9636B', '#64D9D1']), 
              row=1, col=2)

fig.update_layout(height=600, width=800, title_text="Negative vs Postive vs Neutral", template='plotly_white')

fig.show()

In [58]:
string = ' '.join(data.query('sentiment == 1')['clean_text'])

wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(string.lower())
fig = px.imshow(wordcloud)
fig.update_layout(title_text='Negative Reviews')

In [59]:
string = ' '.join(data.query('sentiment == 0')['clean_text'])

wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(string.lower())
fig = px.imshow(wordcloud)
fig.update_layout(title_text='Positive reviews')

In [9]:
import transformers
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding
from tensorflow.keras.layers import LSTM, GRU, Conv1D, SpatialDropout1D

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import activations
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.constraints import *
from tensorflow.keras.initializers import *
from tensorflow.keras.regularizers import *

### Bert Tokenizer

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

save_path = '/kaggle/working/distilbert_base_uncased/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
tokenizer.save_pretrained(save_path)

### Encoding

In [11]:
from sklearn.model_selection import train_test_split

X, y = data['clean_text'].values,data['sentiment'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=111)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.125, 
                                                stratify=y_test, random_state=111)

In [12]:
X_train_tokens = tokenizer(list(X_train), padding='max_length', truncation=True, return_tensors="tf")
X_train_token_ids = X_train_tokens['input_ids']

X_test_tokens = tokenizer(list(X_test), padding='max_length', truncation=True, return_tensors="tf")
X_test_token_ids = X_test_tokens['input_ids']

X_val_tokens = tokenizer(list(X_val), padding='max_length', truncation=True, return_tensors="tf")
X_val_token_ids = X_val_tokens['input_ids']

In [15]:
print(X_test_token_ids.shape)
print(X_train_token_ids.shape)

## Modelling

#### Loading the DistilBert Transformer & creating word embeddings

In [13]:
# Embedding:
transformer = transformers.TFDistilBertModel.\
    from_pretrained('distilbert-base-uncased')
embed = transformer.weights[0].numpy()

print('Vocab : ', np.shape(embed)[0], 'Hidden States/Embed vector size :', np.shape(embed)[1])

In [14]:
MAX_LEN = 512
BATCH_SIZE = 12
STEPS_PER_EPOCH = X_train_token_ids.shape[0] // BATCH_SIZE

### Callback

In [18]:
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',  
                                    factor=0.3, patience=2, 
                                    verbose=1, mode='auto', 
                                    epsilon=0.0001, cooldown=1, min_lr=0.000001)

### Helper functions

In [39]:
def visualize_model_preds(y_pred, indices=[0, 1, 2, 3]):

    for idx, i in enumerate(indices):
        if y_test[i] == 0:
            label = "Non-toxic"
            color = f'{Fore.GREEN}'
            symbol = '\u2714'
        else:
            label = "Toxic"
            color = f'{Fore.RED}'
            symbol = '\u2716'

        print('{}{} {}'.format(color, str(idx+1) + ". " + label, symbol))
        print(f'{Style.RESET_ALL}')

        print(X_test[idx]); print("")
        fig = go.Figure()
        if y_test[i] == 1:
            yl = [1 - y_pred[i], y_pred[i]]
            
        else:
            yl = [1 - y_pred[i], y_pred[i]]

        fig.add_trace(go.Bar(x=['Positive', 'Negative'], y=yl, marker=dict(color=["seagreen", "indianred"])))
        fig.update_traces(name=X_test[idx])
        fig.update_layout(xaxis_title="Labels", yaxis_title="Probability", template="plotly_white", title_text="Predictions for validation comment #{}".format(idx+1))
        fig.show()
        
def plot_cm(y_true, y_pred, title, figsize=(7,6)):
    y_pred = y_pred.round().astype(int)
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    plt.title(title)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)

### 1. Vanilla neural network

In [23]:
def vanilla_model():

    input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")


    embedding = Embedding(np.shape(embed)[0], np.shape(embed)[1],
                              input_length=MAX_LEN, weights=[embed],
                              trainable=False)(input_word_ids)

    x = K.sum(embedding, axis=2)
    x = Dense(512, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    

    model = Model(inputs=input_word_ids, outputs=out)

    model.compile(Adam(lr=0.001), 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])

    return model

In [24]:
van_model = vanilla_model()

train_history = van_model.fit(
    X_train_token_ids, y_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=(X_val_token_ids, y_val),
    epochs=7,
    callbacks = [reduceLROnPlat]
)

#### Evaluate & predict

In [25]:
van_model.evaluate(X_test_token_ids, y_test)

y_pred = van_model.predict(X_test_token_ids)

### Classifying with probabilty threshold = 0.5

In [26]:
y_preds = y_pred > 0.5
y_preds = np.where(y_preds == True, 1, 0)

In [27]:
plot_cm(y_test, y_preds, 'Threshold=0.5')

In [28]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_preds))

Very Poor Performance acheived from Vanilla network with all the positive classes missclassified.

We will be adjusting the probabilty threshold using G-Means

### Probability Threshold Moving : Finding the best Threshold

We have to search a range of threshold values in order to find the best threshold.
In some cases, the optimal threshold can be calculated directly. Tuning or shifting the decision threshold in order to accommodate the broader requirements of the classification problem is generally referred to as threshold-moving, threshold-tuning, or simply thresholding.

#### ROC curve

In [29]:
from sklearn.metrics import roc_curve, precision_recall_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred.reshape(y_test.shape[0]))
gmeans = np.sqrt(tpr * (1-fpr))

# locate the index of the largest g-mean
ix = np.argmax(gmeans)

print('Best Threshold=%f, G-mean=%.3f' % (thresholds[ix], gmeans[ix]))

In [30]:
y_preds = y_pred > thresholds[ix]
y_preds = np.where(y_preds == True, 1, 0)

In [31]:
plot_cm(y_test, y_preds, f'Threshold = {thresholds[ix]}')

Great improvement in the Positve class using Probabilty Threshold moving , but vice versa for Negative class

### 2. Modelling with CNN

In [32]:
def build_cnn_model(max_len=MAX_LEN):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    
    embedding = Embedding(np.shape(embed)[0], np.shape(embed)[1],
                          input_length=max_len, weights=[embed],
                          trainable=False)(input_word_ids)
    
    embedding = SpatialDropout1D(0.3)(embedding)
    conv_1 = Conv1D(64, 2)(embedding)
    conv_2 = Conv1D(64, 3)(embedding)
    conv_3 = Conv1D(64, 4)(embedding)
    conv_4 = Conv1D(64, 5)(embedding)
    
    maxpool_1 = GlobalAveragePooling1D()(conv_1)
    maxpool_2 = GlobalAveragePooling1D()(conv_2)
    maxpool_3 = GlobalAveragePooling1D()(conv_3)
    maxpool_4 = GlobalAveragePooling1D()(conv_4)
    conc = concatenate([maxpool_1, maxpool_2, maxpool_3, maxpool_4], axis=1)

    conc = Dense(64, activation='relu')(conc)
    conc = Dense(1, activation='sigmoid')(conc)
    
    model = Model(inputs=input_word_ids, outputs=conc)
    
    model.compile(Adam(lr=0.01), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    return model

In [33]:
cnn_model = build_cnn_model()

train_history = cnn_model.fit(
    X_train_token_ids, y_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=(X_val_token_ids, y_val),
    epochs=10,
    callbacks = [reduceLROnPlat]
)

### Poor performance from CNN model as well

### 3. LSTM + Attention Layer

In [34]:
class AttentionWeightedAverage(Layer):

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [35]:
def build_lstm_model(max_len=MAX_LEN):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    
    embed = transformer.weights[0].numpy()
    embedding = Embedding(np.shape(embed)[0], np.shape(embed)[1],
                          input_length=max_len, weights=[embed],
                          trainable=False)(input_word_ids)
    
    embedding = SpatialDropout1D(0.3)(embedding)
    lstm_1 = LSTM(128, return_sequences=True)(embedding)
    lstm_2 = LSTM(128, return_sequences=True)(lstm_1)
    
    attention = AttentionWeightedAverage()(lstm_2)
    conc = Dense(64, activation='relu')(attention)
    conc = Dense(1, activation='sigmoid')(conc)
    
    model = Model(inputs=input_word_ids, outputs=conc)
    
    model.compile(Adam(lr=0.01), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    return model

In [36]:
lstm_model = build_lstm_model()

train_history = lstm_model.fit(
    X_train_token_ids, y_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=(X_val_token_ids, y_val),
    epochs=6,
    callbacks = [reduceLROnPlat]
)

The models using CNN, LSTM & Vanilla network did not perform good at all, these models could not learn the Dataset and performed somewhat same.

### 4. Bert Transformer

In [16]:
bert_transformer = transformers.TFDistilBertModel.\
    from_pretrained('distilbert-base-uncased')

In [17]:
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')
y_val = y_val.astype('int32')

In [19]:
def build_bert_model(maxlen=MAX_LEN):
    input_ids = Input(shape=(maxlen,), dtype=tf.int32, name="input_word_ids")
    embeddings = bert_transformer(input_ids)[0]

    cls_token = embeddings[:, 0, :]
    x = Dense(maxlen, activation="relu")(cls_token)
    x = Dropout(0.5)(x)
    out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_ids, outputs=out)

    model.compile(Adam(learning_rate=1.5e-5), 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])
    
    return model

In [20]:
model = build_bert_model()

train_history = model.fit(
    X_train_token_ids, y_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=(X_val_token_ids, y_val),
    epochs=2
)

In [21]:
y_pred = model.predict(X_test_token_ids)

In [29]:
y_pred = y_pred.reshape(y_pred.shape[0])

In [34]:
y_pred[:10]

In [38]:
visualize_model_preds(y_pred)

In [55]:
y_preds = y_pred > 0.5
y_preds = np.where(y_preds == True, 1, 0)

### Accuracy

In [56]:
old_accuracy = accuracy_score(y_preds, y_test)
old_accuracy

In [49]:
plot_cm(y_test, y_preds, 'Threshold=0.5')

### Finding the optimal threshold using ROC-AUC

In [50]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
gmeans = np.sqrt(tpr * (1-fpr))

# locate the index of the largest g-mean
ix = np.argmax(gmeans)

print('Best Threshold=%f, G-mean=%.3f' % (thresholds[ix], gmeans[ix]))

In [51]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.plot([0,1], [0,1], linestyle='--', label='No Skill') 

plt.plot(fpr, tpr, marker='.', label='Logistic') 
plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best') 
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.show()

### Classifying using optimal threshold

In [58]:
y_preds = (y_pred > thresholds[ix])
y_preds = np.where(y_preds == True, 1, 0)

In [63]:
new_accuracy = accuracy_score(y_preds, y_test)


print(f'Percent increase in accuracy after threshold moving = {((new_accuracy - old_accuracy)/old_accuracy) * 100}')

In [64]:
plot_cm(y_test, y_pred, 'Optimal Threshold')